# SCENIC Protocol - **DEBUGGING** - Mouse brain data set


__Author:__ Bram Van de Sande

__Date:__ 6 AUG 2019

__Outline:__ Acquistion and cleaning of selected scRNAseq data sets.

_Experiments:_

| Accession ID | Cancer type | 
| ------------- | ----------- | 
| GSE60361 | Mouse brain |

https://pyscenic.readthedocs.io/en/latest/tutorial.html

# 1. GRNBOOST2 test (Network inference)

## A. Dask connection and package import

In [1]:
from dask.distributed import Client, progress
c = Client()
c

<Client: 'tcp://10.42.5.28:8786' processes=4 threads=24, memory=37.25 GiB>

In [2]:
progress

<function distributed.diagnostics.progressbar.progress(*futures, notebook=None, multi=True, complete=True, **kwargs)>

In [3]:
import os
import glob
import pickle
import pandas as pd
import numpy as np

from dask.diagnostics import ProgressBar

from arboreto.utils import load_tf_names
from arboreto.algo import grnboost2

from ctxcore.rnkdb import FeatherRankingDatabase as RankingDatabase
from pyscenic.utils import modules_from_adjacencies, load_motifs
from pyscenic.prune import prune2df, df2regulons
from pyscenic.aucell import aucell

#import seaborn as sns
from IPython.core.debugger import set_trace

## B. Input file pre-processing

In [4]:
PROJECT = "/home/jovyan/work/dask_k8s/project"
DATA_FOLDER="/tmp"
RESOURCES_FOLDER="/resources"
#DATABASE_FOLDER = "/databases/"
DATABASE_FOLDER = PROJECT+"/databases/"
SCHEDULER="myrelease-dask-scheduler:8786"
#DATABASES_GLOB = "."+os.path.join(PROJECT, DATABASE_FOLDER, "mm9-*.mc9nr.genes_vs_motifs.rankings.feather")
DATABASES_GLOB = os.path.join(DATABASE_FOLDER, "mm9-*.mc9nr.genes_vs_motifs.rankings.feather")
MOTIF_ANNOTATIONS_FNAME = "."+os.path.join(PROJECT, RESOURCES_FOLDER, "motifs-v9-nr.mgi-m0.001-o0.0.tbl")
MM_TFS_FNAME = "."+os.path.join(PROJECT, RESOURCES_FOLDER, 'mm_mgi_tfs.txt')
SC_EXP_FNAME = "."+os.path.join(PROJECT, RESOURCES_FOLDER, "GSE60361_C1-3005-Expression.txt")
REGULONS_FNAME = os.path.join(PROJECT, DATA_FOLDER, "regulons.p")
MOTIFS_FNAME = os.path.join(PROJECT, DATA_FOLDER, "motifs.csv")

In [5]:
os.listdir("/home/jovyan/work/dask_k8s/project/databases/")

['mm9-500bp-upstream-7species.mc9nr.genes_vs_motifs.rankings.feather',
 'hidden',
 'mm9-tss-centered-10kb-7species.mc9nr.genes_vs_motifs.rankings.feather']

In [6]:
#%debug

In [7]:
SC_EXP_FNAME

'./resources/GSE60361_C1-3005-Expression.txt'

In [8]:
MOTIF_ANNOTATIONS_FNAME

'./resources/motifs-v9-nr.mgi-m0.001-o0.0.tbl'

In [9]:
DATABASES_GLOB

'/home/jovyan/work/dask_k8s/project/databases/mm9-*.mc9nr.genes_vs_motifs.rankings.feather'

In [6]:
#Load expression data
def load_matrix():
    #set_trace()
    matrix = pd.read_csv(SC_EXP_FNAME, sep='\t', header=0, index_col=0).T
    return matrix

ex_matrix = load_matrix()
ex_matrix.shape

(3005, 19972)

In [7]:
#Reducing matrix
ex_matrix = ex_matrix.iloc[:, :12000]

In [12]:
#df memory usage
	
ex_matrix.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Index: 3005 entries, 1772071015_C02 to 1772058148_F03
Columns: 19972 entries, Tspan12 to Gm20738_loc3
dtypes: int64(19972)
memory usage: 458.1 MB


In [8]:
#Load tf names
tf_names = load_tf_names(MM_TFS_FNAME)

In [14]:
len(tf_names)

1721

In [15]:
type(tf_names)

list

In [16]:
c

<Client: 'tcp://10.42.5.21:8786' processes=24 threads=24, memory=51.41 GiB>

## C. Setting the start timestamp

In [17]:
from datetime import datetime
currentDateAndTime = datetime.now()
currentDateAndTime = currentDateAndTime.strftime("%Y_%m_%d_%H-%M-%S")
print(currentDateAndTime)

2023_02_25_21-50-51


In [18]:
#Start date 
!date -u +"%c"

Sat 25 Feb 2023 09:50:51 PM UTC


In [19]:
#Start date Unix
!date +"%s"

1677361852


## D. GRNboost2 run

### i. Running

In [20]:
#Run GRNboost2 in debug mode
#
#if __name__ == '__main__':
#    #c = Client()
#    from dask.distributed import performance_report
#    with performance_report(filename="dask-report_%s.html"%(currentDateAndTime)):
#        def grn(var1, var2, var3):
#            set_trace() #Activating debug mode
#            return grnboost2(var1, verbose=var2, client_or_address=var3)
#        adjacencies2 = grn(ex_matrix, var2=True, var3=c)

In [21]:
#GRNBoost2 in normal mode
if __name__ == '__main__':
    from dask.distributed import performance_report
    with performance_report(filename="dask-report_%s.html"%(currentDateAndTime)):
       adjacencies = grnboost2(ex_matrix, tf_names=tf_names, verbose=True, client_or_address=c)

preparing dask client
parsing input
creating dask graph
24 partitions
computing dask graph
not shutting down client, client was created externally
finished


In [22]:
#end date 
!date -u +"%c"

Sat 25 Feb 2023 10:57:38 PM UTC


### ii. Saving file

In [24]:
adjacencies.head()

,TF,target,importance
65,Gm14305,OTTMUSG00000016609_loc4,399.775712
65,Gm14305,OTTMUSG00000016609_loc3,242.122851
868,Rpl35,Rpl32,148.716645
868,Rpl35,Rpl18a,145.586963
82,Mef2c,Camk2n1,142.321890


In [23]:
#Getting adjacencies dimensions
adjacencies.shape

(5915026, 3)

In [5]:
#Dumping adjacencies for later
adjacencies.to_csv("intermediate_results/adjacencies_full-matrix.csv")

NameError: name 'adjacencies' is not defined

### iii. Re-loading file

In [9]:
adjacencies_dump = pd.read_csv("intermediate_results/adjacencies_12000.csv", index_col=0)

In [10]:
adjacencies_dump.head()

,TF,target,importance
6248,Zfp433_loc2,Zfp433_loc1,664.614695
7831,Srp54b_loc2,Srp54b_loc1,599.162420
7829,1700047I17Rik2_loc2,Fam177a_loc1,581.545982
7827,Fam177a_loc1,1700047I17Rik2_loc2,571.602001
9251,Gm16677_loc2,Gm16677_loc1,560.672568


In [11]:
adjacencies_dump.shape

(6986837, 3)

# 2. Regulon prediction

**Note**: The nodes of the clusters need to have access to a shared network drive on which the ranking databases are stored.

## A. Obtaining the modules

In [12]:
#Obtaining module list
modules = list(modules_from_adjacencies(adjacencies_dump, ex_matrix))


2023-02-27 21:23:13,736 - pyscenic.utils - INFO - Calculating Pearson correlations.

2023-02-27 21:23:16,280 - pyscenic.utils - WARNING - Note on correlation calculation: the default behaviour for calculating the correlations has changed after pySCENIC verion 0.9.16. Previously, the default was to calculate the correlation between a TF and target gene using only cells with non-zero expression values (mask_dropouts=True). The current default is now to use all cells to match the behavior of the R verision of SCENIC. The original settings can be retained by setting 'rho_mask_dropouts=True' in the modules_from_adjacencies function, or '--mask_dropouts' from the CLI.
	Dropout masking is currently set to [False].

2023-02-27 21:26:07,650 - pyscenic.utils - INFO - Creating modules.


In [15]:
#Checking object type
type(modules)

list

## B. Inspecting the Feather Ranking database object

In [16]:
#Loading ranking databases
db_fnames = glob.glob(DATABASES_GLOB)
def name(fname):
    return os.path.splitext(os.path.basename(fname))[0]
dbs = [RankingDatabase(fname=fname, name=name(fname)) for fname in db_fnames]
dbs

[FeatherRankingDatabase(name="mm9-500bp-upstream-7species.mc9nr.genes_vs_motifs.rankings"),
 FeatherRankingDatabase(name="mm9-tss-centered-10kb-7species.mc9nr.genes_vs_motifs.rankings")]

In [17]:
#Finding out what type of object is created by the RankingDatabase() function
type(dbs[0])

ctxcore.rnkdb.FeatherRankingDatabase

In [18]:
#Finding what is stored in db_fnames
db_fnames

['/home/jovyan/work/dask_k8s/project/databases/mm9-500bp-upstream-7species.mc9nr.genes_vs_motifs.rankings.feather',
 '/home/jovyan/work/dask_k8s/project/databases/mm9-tss-centered-10kb-7species.mc9nr.genes_vs_motifs.rankings.feather']

In [19]:
#Finding the name() function output
name(db_fnames[0])

'mm9-500bp-upstream-7species.mc9nr.genes_vs_motifs.rankings'

## C. Debugging database loading

In [29]:
#Debugging the load method
def debug_database_load(module):
    set_trace()
    return dbs[0].load(module)
#    
#dbs[0].load(modules[0])
debug_database_load(modules[0])

> /tmp/ipykernel_1001/4019392965.py(4)debug_database_load()
      2 def debug_database_load(module):
      3     set_trace()
----> 4     return dbs[0].load(module)
      5 #
      6 #dbs[0].load(modules[0])



ipdb>  s


--Call--
> /opt/conda/lib/python3.8/site-packages/ctxcore/rnkdb.py(128)load()
    126         )
    127 
--> 128     def load(self, gs: GeneSignature) -> pd.DataFrame:
    129         # For some genes in the signature there might not be a rank available in the database.
    130         gene_set = self.geneset.intersection(set(gs.genes))



ipdb>  s


> /opt/conda/lib/python3.8/site-packages/ctxcore/rnkdb.py(130)load()
    128     def load(self, gs: GeneSignature) -> pd.DataFrame:
    129         # For some genes in the signature there might not be a rank available in the database.
--> 130         gene_set = self.geneset.intersection(set(gs.genes))
    131 
    132         return self.ct_db.subset_to_pandas(



...

> /opt/conda/lib/python3.8/site-packages/ctxcore/ctdb.py(789)subset_to_pandas()
    787         # Fetch scores or rankings for input region IDs or gene IDs from cisTarget database file for region IDs or
    788         # gene IDs which were not prefetched in previous calls.
--> 789         self.prefetch(region_or_gene_ids=region_or_gene_ids, engine=engine, sort=True)
    790 
    791         if not self.df_cached:



ipdb>  n


FileNotFoundError: [Errno 2] Failed to open local file '/home/jovyan/work/dask_k8s/project/databases/mm9-500bp-upstream-7species.mc9nr.genes_vs_motifs.rankings.feather'. Detail: [errno 2] No such file or directory
> /opt/conda/lib/python3.8/site-packages/ctxcore/ctdb.py(789)subset_to_pandas()
    787         # Fetch scores or rankings for input region IDs or gene IDs from cisTarget database file for region IDs or
    788         # gene IDs which were not prefetched in previous calls.
--> 789         self.prefetch(region_or_gene_ids=region_or_gene_ids, engine=engine, sort=True)
    790 
    791         if not self.df_cached:



ipdb>  c


FileNotFoundError: [Errno 2] Failed to open local file '/home/jovyan/work/dask_k8s/project/databases/mm9-500bp-upstream-7species.mc9nr.genes_vs_motifs.rankings.feather'. Detail: [errno 2] No such file or directory

In [28]:
%debug

> /home/jovyan/work/dask_k8s/project/pyarrow/error.pxi(113)pyarrow.lib.check_status()



ipdb>  p module


*** NameError: name 'module' is not defined


ipdb>  q


## D. Running prune2df

### i. Normal Run

In [81]:
# Calculate a list of enriched motifs and the corresponding target genes for all modules.
with ProgressBar():
    set_trace()
    df = prune2df(dbs, modules, MOTIF_ANNOTATIONS_FNAME, client_or_address=c)

KeyboardInterrupt: 

### ii. Debug run

In [ ]:
%xmode verbose
%pdb on
def prune_debug():
    with ProgressBar():
        #set_trace()
        return prune2df(dbs, modules, MOTIF_ANNOTATIONS_FNAME, client_or_address=c)
    
df = prune_debug()

Exception reporting mode: Verbose
Automatic pdb calling has been turned ON


/opt/conda/lib/python3.8/site-packages/distributed/worker.py:2988: UserWarning: Large object of size 4.64 MiB detected in task graph: 
  (["('from-delayed-c4019a539b12fc6da49b4942189d12b4 ... b4', 82669)"],)
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  warnings.warn(


ipdb>  s


In [33]:
# Create regulons from this table of enriched motifs.
regulons = df2regulons(df)

NameError: name 'df' is not defined

2023-02-22 01:17:07,149 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
2023-02-22 01:17:07,152 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


In [ ]:
# Save the enriched motifs and the discovered regulons to disk.
df.to_csv(MOTIFS_FNAME)
with open(REGULONS_FNAME, "wb") as f:
    pickle.dump(regulons, f)